In [289]:
import pandas as pd
import numpy as np

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)


numpy version: 1.24.2
pandas version: 1.5.3


## Load data

In [227]:
# carregando os dados
vendas = pd.read_csv('data/vendas.csv', skiprows=3, header=1, usecols=[2,3,4,5,6,7,8,9])
consumidores = pd.read_csv('data/consumidores.csv')
produto = pd.read_csv('data/produto.csv')
lojas = pd.read_csv('data/lojas.csv')

## Merge dataframes
Possíveis uniões

* vendas e consumidores podem ser unidos pela coluna 'ClientID'.
* vendas e lojas podem ser unidos pela coluna 'StoreID'.
* vendas e produto podem ser unidos pela coluna 'ProductID'.


In [169]:
print("Vendas columns:", vendas.columns)
print("Consumidores columns:", consumidores.columns)
print("Lojas columns:", lojas.columns)
print("Produtos columns:", produto.columns)

Vendas columns: Index(['ID', 'ProductID', 'ClientID', 'Discount', 'UnitPrice', 'Quantity',
       'StoreID', 'Date'],
      dtype='object')
Consumidores columns: Index(['ID', 'City', 'State', 'DateOfBirth', 'Sex'], dtype='object')
Lojas columns: Index(['ID', 'Name', 'State'], dtype='object')
Produtos columns: Index(['ID', 'Name', 'Size'], dtype='object')


In [247]:
columns_to_suffix = ['City','State']
consumidores = consumidores.rename(columns={col: f'{col}_cons' for col in columns_to_suffix})

In [246]:
columns_to_suffix = ['Name','State']
lojas = lojas.rename(columns={col: f'{col}_loj' for col in columns_to_suffix})

In [245]:
columns_to_suffix = ['Name']
produto = produto.rename(columns={col: f'{col}_pro' for col in columns_to_suffix})

In [238]:
vendas_consumidor = pd.merge(vendas, consumidores, left_on='ClientID', right_on='ID',suffixes=("_vend", "_cons"))
vendas_consumidor = vendas_consumidor.drop('ID_cons', axis=1)

vendas_lojas = pd.merge(vendas, lojas, left_on='StoreID', right_on='ID',suffixes=("_vend", "_loja"))
vendas_lojas = vendas_lojas.drop('ID_loja', axis = 1)

vendas_produto = pd.merge(vendas, produto, left_on='ProductID', right_on='ID',suffixes=("_vend", "_prod"))
vendas_produto =  vendas_produto.drop('ID_prod', axis = 1)

combined_df = vendas_consumidor.merge(vendas_lojas, on=['ID_vend', 'ProductID', 'ClientID', 'Discount', 'UnitPrice', 'Quantity', 'StoreID', 'Date'])
combined_df = combined_df.merge(vendas_produto, on=['ID_vend', 'ProductID', 'ClientID', 'Discount', 'UnitPrice', 'Quantity', 'StoreID', 'Date'])

In [241]:
combined_df.head()

,ID_vend,ProductID,ClientID,Discount,UnitPrice,Quantity,StoreID,Date,City_cons,State_cons,DateOfBirth,Sex,Name_loj,State_loj,Name_pro,Size
0,80260d682079b6090c8285b398c50d97,002552c0663708129c0019cc97552d7d3,14001,"0,08","249,2",1,4,12/10/2018,Curitiba,PR,6/28/1985,Homem,Curitiba,PR,Tempestade,G
1,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,"0,1","162,4",1,4,4/25/2019,Curitiba,PR,6/28/1985,Homem,Curitiba,PR,Thanos,G
2,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,"0,1","194,6",1,4,7/17/2019,Curitiba,PR,6/28/1985,Homem,Curitiba,PR,Capitão América,P
3,7180475b60fe6bbbde424d39704a1177,001b72dfd63e9833e8c02742adf472e33,14002,"0,1","201,6",2,1,4/10/2018,Florianópolis,SC,1/10/1987,Homem,Florianópolis,SC,Wolverine,G
4,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,"0,1",406,1,1,1/21/2019,Florianópolis,SC,1/10/1987,Homem,Florianópolis,SC,Naruto,G


In [260]:
# substituir vígula por ponto para se enquadrar no padrão internacional
combined_df.Discount = combined_df.Discount.str.replace(',', '.')
combined_df.UnitPrice = combined_df.UnitPrice.str.replace(',', '.')

## Convert data types

In [264]:
def convert_columns_types(df):
    # Dicionário com os tipos de colunas
    column_types = {
        'date': ['Date', 'DateOfBirth'],
        'numeric': ['Discount', 'UnitPrice', 'Quantity'],
        'category': ['ID_vend', 'ProductID', 'ClientID','StoreID', 'City_cons', 'State_cons','Sex','Name_loj', 'State_loj', 'Name_pro', 'Size']
    }

    # Função para converter colunas de data
    def convert_date_columns(df, columns):
        for col in columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], format='%m/%d/%Y')
        return df

    # Função para converter colunas numéricas
    def convert_numeric_columns(df, columns):
        for col in columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col])
        return df

    # Função para converter colunas de categoria
    def convert_category_columns(df, columns):
        for col in columns:
            if col in df.columns:
                df[col] = df[col].astype('category')
        return df

    # Converter colunas de acordo com os tipos definidos no dicionário
    df = convert_date_columns(df, column_types['date'])
    df = convert_numeric_columns(df, column_types['numeric'])
    df = convert_category_columns(df, column_types['category'])

    return df

# Exemplo de uso da função
combined_df_converted = convert_columns_types(combined_df.copy())
combined_df_converted


,ID_vend,ProductID,ClientID,Discount,UnitPrice,Quantity,StoreID,Date,City_cons,State_cons,DateOfBirth,Sex,Name_loj,State_loj,Name_pro,Size
0,80260d682079b6090c8285b398c50d97,002552c0663708129c0019cc97552d7d3,14001,0.08,249.2,1,4,2018-12-10,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Tempestade,G
1,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,0.10,162.4,1,4,2019-04-25,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Thanos,G
2,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,0.10,194.6,1,4,2019-07-17,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Capitão América,P
3,7180475b60fe6bbbde424d39704a1177,001b72dfd63e9833e8c02742adf472e33,14002,0.10,201.6,2,1,2018-04-10,Florianópolis,SC,1987-01-10,Homem,Florianópolis,SC,Wolverine,G
4,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,0.10,406.0,1,1,2019-01-21,Florianópolis,SC,1987-01-10,Homem,Florianópolis,SC,Naruto,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32748,b155e2a8589d40bc9c0381ee48e3204f,002ec297b1b00fb9dde7ee6ac24b67712,28694,0.08,218.4,1,4,2018-12-02,Curitiba,PR,1982-10-22,Homem,Curitiba,PR,Bazinga,M
32749,abb71d0110c4be1a1a0c52f929cab4a3,002d4ea7c04739c130bb74d7e7cd16943,28694,0.08,210.0,1,4,2018-12-05,Curitiba,PR,1982-10-22,Homem,Curitiba,PR,Iron Man,G
32750,db9a034f34174edd577b9f64cc982952,001795ec6f1b187d37335e1c4704762e3,28694,0.08,148.4,1,4,2019-08-01,Curitiba,PR,1982-10-22,Homem,Curitiba,PR,Mulher-Maravilha,G
32751,ee00fa5093eb96d8573403b382fb60a8,002ec297b1b00fb9dde7ee6ac24b67713,28694,0.10,218.4,2,4,2019-11-25,Curitiba,PR,1982-10-22,Homem,Curitiba,PR,Bazinga,G


## Create new variables

In [ ]:
combined_df_converted['Revenue'] = combined_df_converted['UnitPrice'] * (1 - combined_df_converted['Discount']) * combined_df_converted['Quantity']

In [ ]:
combined_df_converted['Year'] = combined_df_converted['Date'].dt.year
combined_df_converted['Month'] = combined_df_converted['Date'].dt.month
combined_df_converted['Day'] = combined_df_converted['Date'].dt.day

In [270]:
combined_df_converted['Age'] = ((combined_df_converted['Date'] - combined_df_converted['DateOfBirth']) / pd.Timedelta(days=365)).astype(int)

In [276]:
combined_df_converted.head()

,ID_vend,ProductID,ClientID,Discount,UnitPrice,Quantity,StoreID,Date,City_cons,State_cons,DateOfBirth,Sex,Name_loj,State_loj,Name_pro,Size,Revenue,Age
0,80260d682079b6090c8285b398c50d97,002552c0663708129c0019cc97552d7d3,14001,0.08,249.2,1,4,2018-12-10,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Tempestade,G,229.264,33
1,3275736da1234f55d52bdf09d86b93bc,001b237c0e9bb435f2e54071129237e93,14001,0.10,162.4,1,4,2019-04-25,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Thanos,G,146.160,33
2,f3513eec8f1434b5b7e04ddf3598df40,00066f42aeeb9f3007548bb9d3f33c381,14001,0.10,194.6,1,4,2019-07-17,Curitiba,PR,1985-06-28,Homem,Curitiba,PR,Capitão América,P,175.140,34
3,7180475b60fe6bbbde424d39704a1177,001b72dfd63e9833e8c02742adf472e33,14002,0.10,201.6,2,1,2018-04-10,Florianópolis,SC,1987-01-10,Homem,Florianópolis,SC,Wolverine,G,362.880,31
4,76600853a200fe87e141adae6b35674c,00210e41887c2a8ef9f791ebc780cc363,14002,0.10,406.0,1,1,2019-01-21,Florianópolis,SC,1987-01-10,Homem,Florianópolis,SC,Naruto,G,365.400,32


## Save

In [279]:
combined_df_converted.to_parquet('data/cleaned.parquet')